In [12]:
import pandas as pd 

In [13]:
pdf_valuations = pd.read_csv('../data/input/player_valuations.csv')[["player_id", "date", "market_value_in_eur"]]
pdf_players = pd.read_csv('../data/input/players.csv')[["player_id", "name", "date_of_birth", "position", "sub_position", "contract_expiration_date", "current_club_name", "current_club_domestic_competition_id" ]]

In [15]:
pdf_joined = pdf_players.merge(pdf_valuations, on="player_id").query("current_club_domestic_competition_id == 'GB1'")
pdf_joined["age"] = (pd.to_datetime(pdf_joined["date"]) - pd.to_datetime(pdf_joined["date_of_birth"])).dt.days // 365
pdf_joined["year"] = pd.to_datetime(pdf_joined["date"]).dt.year
pdf_joined["month"] = pd.to_datetime(pdf_joined["date"]).dt.month


In [17]:
pdf_joined["market_value_in_million_eur"] = pdf_joined["market_value_in_eur"] / 1000000
pdf_mvp = pdf_joined[["name", "player_id", "date_of_birth", "date", "market_value_in_million_eur", "age", "year", "month" ]]



In [18]:
# Filter training and test sets
train = pdf_mvp[pdf_mvp["year"] <= 2022].sort_values("date", ascending=False)
test = pdf_mvp[(pdf_mvp["year"] >= 2023)].sort_values("date", ascending=False)
pdf_train_agg= train.query("year == 2022").groupby(["player_id", "name"])[["date_of_birth", "market_value_in_million_eur", "age"]].first().reset_index()
pdf_test_agg = test.query("year == 2023").groupby(["player_id", "name"])[["date_of_birth", "market_value_in_million_eur", "age"]].last().reset_index()
pdf_test_train = pdf_train_agg.merge(pdf_test_agg, on=["player_id", "name"], suffixes=("_train", "_test"))

In [19]:
pdf_test_train.to_csv("../data/intermediate/traditional_model_data_prep.csv", index=False)
